# Complex semidefinite programming

Finally, we move to applications of semidefinite programming in quantum
information! Here, one typically considers linear matrix inequalities concerning
*Hermitian* as opposed to symmetric matrices.

## Hermitian positive semidefiniteness

Let $A \in \mathbb{C}^{n \times n}$ be a square matrix with complex entries. We
define its *adjoint* (also known as *Hermitian transpose* or *conjugate
transpose*) as $A^\dagger = \overline{A^T} = \overline{A}^T$, where
$\overline{B} = \Re(B) - \Im(B)$ denotes the complex conjugate of a matrix $B$.
Other common notations for the adjoint are $A^H$ or $A^*$, though the latter is
sometimes used for the complex conjugate instead.

We say that $A$ is *Hermitian* if $A^\dagger = A$, or, equivalently, if its real
part is symmetric, $\Re(A^\dagger) = \Re(A)$, while its imaginary part is
skew-symmetric, $\Im(A^\dagger) = -\Im(A)$.

If $A$ is indeed Hermitian, then from $\Im(A^T) = -\Im(A)$, it follows that
$\operatorname{diag}(\Im(A)) = \mathbf{0}$, thus $\operatorname{Tr}(A) \in
\mathbb{R}$. This makes the trace of a Hermitian matrix a plausible target for
optimization. A common "source" of Hermitian matrices in applications are
products of the form $B^\dagger B$ or $B B^\dagger$:

---

<div class="alert alert-info">

   **Task 4.1:** Prove that $B^\dagger B$ (and thus $B B^\dagger$) is Hermitian
   for any $B \in \mathbb{C}^{m \times n}$.

   <details>
   <summary style='display: list-item'>Hint</summary>

   - Option 1: Write ${(B^\dagger B)}_{ij}$ as a sum over row indices $r \leq
     m$ and relate it to ${(B^\dagger B)}_{ji}$.
   - Option 2: Simplify ${(B^\dagger B)}^\dagger$ using the definition of the
     adjoint and suited properties of the transpose and complex conjugate.

   </details>

</div>

---

A Hermitian matrix $A \in \mathbb{C}^{n \times n}$ is *(hermitian) positive
semidefinite* (HPSD), written $A \succeq 0$, if $z^\dagger A z \geq 0$ for all
$z \in \mathbb{C}^{n} \setminus \{0\}$.

In the second notebook, you proved that the outer product $x x^T$ is positive
semidefinite (PSD) for any $x \in \mathbb{R}^n$. Clearly, also $x^T x$ is PSD.
This is more generally true for any complex matrix $B \in \mathbb{C}^{m \times
n}$: Both $B^\dagger B$ and $B B^\dagger$ are HPSD!

Numeric solvers typically do not support complex input at all. However, there is
a straightforward real reformulation of $A \succeq 0$ ([Goemans and Williamson,
2001](#references)) that is implemented in PICOS:

---

<div class="alert alert-info">

   **Bonus task 4.2:** Fill in the gaps to show that a Hermitian matrix $A$ is HPSD if
   and only if some symmetric matrix $\rho(A)$ is PSD.

   <details>
   <summary style='display: list-item'>Detailed instructions</summary>

   - (a) Use a definition of $A \succeq 0$.
   - (b) Expand $z$ into $a + bi$ and $A$ into $\Re(A) + \Im(A)i$. Also apply
     the definition of the adjoint so that only $\cdot^T$ but not
     $\cdot^\dagger$ appears.
   - (c) Multiply out into eight summands. Factor out $i^2 = -1$ so that at most
     one $i$ appears in each summand.
   - (d) Separate the real and imaginary summands. Make use of the skew-symmetry
     of $\Im(A)$ to simplify the imaginary part (you can simplify
     *significantly*).
   - (e) Rewrite the remaining sum as a quadratic form.

   </details>

</div>

<div hidden>

$$
\providecommand{\TODO}{}\renewcommand{\TODO}{{\color{red}{[\ldots]}}}
$$

</div>

$$
    A \succeq 0
    \quad
    \overset{\text{(a)}}\Longleftrightarrow
    \quad
    \forall z = a + bi \in \mathbb{C}^n \setminus \{0\} \colon
    \underbrace{
        \TODO
    }_\star \geq 0
$$

where

$$
\begin{align*}
    \star
    &\overset{\text{(b)}}=
        \TODO
    \\
    &\overset{\text{(c)}}=
        \TODO
    \\
    &\overset{\text{(d)}}=
    \left(
        \TODO
    \right) + \left(
        \TODO
    \right) i \\
    &\overset{\text{(e)}}=
    \begin{pmatrix} a^T & b^T \end{pmatrix}
    \underbrace{\begin{bmatrix}
        \TODO & \TODO \\
        \TODO & \TODO
    \end{bmatrix}}_{\rho(A)}
    \begin{pmatrix} a \\ b \end{pmatrix}.
\end{align*}
$$

From

$$
    0 \neq z \in \mathbb{C}^n
    \quad \Longleftrightarrow \quad
    0 \neq \begin{pmatrix} a \\ b \end{pmatrix} \in \mathbb{R}^n
$$

it follows that

$$
    A \succeq 0
    \quad \Longleftrightarrow \quad
    \rho(A) \succeq 0.
$$

## Problem 4: Quantum channel discrimination

We will use semidefinite programs involving Hermitian matrices to compute the
*completely bounded trace norm*, or *diamond norm*, of a given matrix. This norm
plays a role in the following communication problem.

### Setting

Consider two parties, Alice and Bob, who communicate as follows. First,
**Alice** prepares a pure quantum state

$$
   \varrho_{AR} = \rvert \psi \rangle \langle \psi \lvert = \psi \psi^\dagger
   \qquad
   \text{for}
   \qquad
   \psi \in \mathcal{H}_{AR}
   = \mathcal{H}_A \otimes \mathcal{H}_R
   = \mathbb{C}^{d_A d_R}
   \qquad
   \text{with}
   \qquad
   {\lVert \psi \rVert}_2 = 1
$$

whose subsystems $\varrho_A$ and $\varrho_R$ are possibly entangled. We assume for
simplicity that the subsystem dimensions are equal, i.e. $d_A = d_R$.$^1$ Next,
Alice hands the $A$-subsystem of $\varrho_{AR}$, formally the partial trace $\varrho_A
= \operatorname{Tr}_R(\varrho_{AR})$, to Bob.

**Bob** has on his end two quantum channels, $\mathcal{B}_i \colon
\mathbb{C}^{d_A \times d_A} \to \mathbb{C}^{d_B \times d_B}$, for $i \in
\{1,2\}$. He chooses $i = 1$ with probability $t$ and $i = 2$ with probability
$1-t$, applies the corresponding channel to $\varrho_A$, and hands back the result
$\mathcal{B}_i(\varrho_A)$ to Alice.

After this exchange, **Alice** would like to guess the channel used by Bob by
measuring the modified joint state $(\mathcal{B}_i \otimes
\mathcal{I}_R)(\varrho_{AR})$, where $\mathcal{I}_R = \mathcal{I}_A$ denotes the
identity channel of the $R$-subsystem that she kept to herself. She controls the
state $\psi$ and she knows everything about the above setup apart from the random
outcome $i$ from Bob's biased coin toss.

With what probability can Alice distinguish between $i = 1$ and $i = 2$? If we
denote the random channel by $$\mathcal{D} = t \mathcal{B}_1 - (1-t)
\mathcal{B}_2,$$ then the answer turns out to be

$$
   s^* = \frac{1}{2}\left( 1 + \sup_{\varrho_{AR}} {\lVert (\mathcal{D}
   \otimes \mathcal{I}_A)(\varrho_{AR}) \rVert}_1 \right)
$$

where ${\lVert X \rVert}_1 = \operatorname{Tr}\left( \sqrt{X^\dagger X} \right)$
denotes the *trace norm* of an operator $X$.$^2$ The term

$$
   \sup_{\varrho_{AR}} {\lVert (\mathcal{D} \otimes \mathcal{I}_A)(\varrho_{AR})
   \rVert}_1
   \overset{^3}= \sup_{X \colon {\lVert X \rVert}_1 \leq 1} {\lVert
   (\mathcal{D} \otimes \mathcal{I}_A)(X) \rVert}_1
   % = {\lVert \mathcal{D} \otimes \mathcal{I}_A \rVert}_1
   = {\lVert \mathcal{D} \rVert}_\diamond
$$

is called the *diamond norm* of $\mathcal{D}$. Computing this norm amounts to
maximizing over a convex function, which is a nonconvex problem. But, in this
particular case a conic reformulation as an SDP is possible.

Before we get to this, let's consider also the case where Alice does not have an
auxiliary system $h$ at her disposal but can only choose a quantum state
$\varrho_A$. In this case, the probability for success simplifies to

$$
   q^* = \frac{1}{2}\left( 1 + \sup_{\varrho_A} {\lVert \mathcal{D} (\varrho_A)
   \rVert}_1 \right)
$$

Also here we would like to maximize over a convex function to calculate the
*induced trace norm*$^4$

$$
   \sup_{\varrho_A} {\lVert \mathcal{D} (\varrho_A) \rVert}_1
   = \sup_{X \colon {\lVert X \rVert}_1 \leq 1}
      {\lVert \mathcal{D} (X) \rVert}_1
   = {\lVert \mathcal{D} \rVert}_1.
$$

This time, though, no equivalent convex formulation appears to be known.

<details>
<summary style='display: list-item'>Footnotes</summary>

1. If Alice controls also the auxiliary system dimension $d_R$, then setting
   $d_R = d_A$ is optimal for her.
2. The trace norm is also known as the *nuclear norm* or the *Schatten 1-norm*.
   The matrix root in its definition is well-defined for HPSD matrices.
3. That the supremum over $X$ with ${\lVert X \rVert}_1 \leq 1$ is indeed
   attained by a rank-one density operator is discussed, e.g., by Watrous
   ([2018](#references), Proposition 3.38 and Theorem 3.39).
4. Inconveniently, the induced trace norm and the trace norm use the same
   notation in the literature.

</details>

### The problem

We would like to compute Alice's success probabilities when she is allowed to
make use of the $R$-subsystem versus when she is not. This probability depends
on the channels available to Bob. Here, we assume that Bob chooses between one
of two *Werner-Holevo* channels with equal probability, i.e. $t = \frac{1}{2}$,

$$
\begin{align*}
   \mathcal{B}_1(\varrho) = \frac{1}{d + 1} \left( \operatorname{Tr}(\varrho) I +
   \varrho^T \right),
   \qquad
   \text{and}
   \qquad
   \mathcal{B}_2(\varrho) = \frac{1}{d - 1} \left( \operatorname{Tr}(\varrho) I -
   \varrho^T \right).
\end{align*}
$$

We will need their Choi–Jamiołkowski representations:

$$
\begin{align*}
   \mathcal{J}_{BA}(\mathcal{B}_1) = \frac{1}{d + 1}\left(I_B \otimes I_A +
   S_{BA}\right)
   \qquad
   \text{and}
   \qquad
   \mathcal{J}_{BA}(\mathcal{B}_2) = \frac{1}{d - 1}\left(I_B \otimes I_A -
   S_{BA}\right),
\end{align*}
$$

where $S_{BA}(u \otimes v) = v \otimes u$ for $u, v \in \mathbb{C}^{d_A} =
\mathbb{C}^{d_B}$ is called the *swap operator*.


---

<div class="alert alert-info">

   **Task 4.3:** Write a function that takes as argument a parameter `d`
   representing $d = d_A = d_B$ and outputs the $d^2 \times d^2$ swap operator
   matrix $S_{AB} = S_{BA}$.

   <details>
   <summary style='display: list-item'>Definition of the Kronecker product</summary>

   For two column vectors $u \in \mathbb{C}^a$ and $u \in \mathbb{C}^b$, it is

   $$
   \begin{align*}
      u \otimes v
      = \begin{pmatrix}
         u_1 v \\ \vdots \\ u_a v
      \end{pmatrix}
      = \begin{pmatrix}
         u_1 v_1 \\
         \vdots \\
         u_1 v_b \\
         \vdots \\
         u_a v_1 \\
         \vdots \\
         u_a v_b
      \end{pmatrix}.
   \end{align*}
   $$

   </details>

   <details>
   <summary style='display: list-item'>Recommended hints</summary>

   - The swap operator is a permutation matrix: a binary matrix with exactly
     one $1$ in each column and in each row.
   - If $P_{ij} = 1$ for a permutation matrix $P$, then it "moves" the $j$-th
     row of its operand to the $i$-th row.
   - Consider an index $i$ to $u$ and $j$ to $v$. Where does the element $u_i
     v_j$ end up in $u \otimes v$ and in $v \otimes u$, respectively?
   - You will need to account for the fact that indices in Python start at $0$.

   </details>

   <details>
   <summary style='display: list-item'>More hints</summary>

   For indices starting at $0$:

   - It is ${(u \otimes v)}_{di+j} = u_i v_j = {(v \otimes u)}_{dj+i}$ for all
     $0 \leq i, j < d$ by the definition of the Kronecker product.
   - We want $S {(u \otimes v)}_{dj+i} \overset{!}= {(v \otimes u)}_{dj+i} = {(u
     \otimes v)}_{di+j}$ for all $0 \leq i, j < d$.

   </details>

   <details>
   <summary style='display: list-item'>Python and library documentation</summary>

   - You can use `for`-loops and `range` (see
     [here](https://pynative.com/python-range-function/) for an example).
   - You may use
     [np.zeros](https://numpy.org/doc/stable/reference/generated/numpy.zeros.html)
     to create an all-zero matrix (2D array) `S` to start with, then use
     `S[i,j] = x` to set $S_{ij} = x$.
   - It is also possible to use a sparse matrix type instead of a dense NumPy
     array, as $S$ is a sparse matrix. Sparse types understood by PICOS are
     those of [SciPy](https://docs.scipy.org/doc/scipy/reference/sparse.html)
     and [CVXOPT](https://cvxopt.org/userguide/matrices.html#sparse-matrices).

   </details>
</div>

In [ ]:
import numpy as np
import picos as pc

def swap(d):
    # TODO: Return a matrix S such that S·(u⊗v) = v⊗u for d-vectors u and v.
    # S = 
    return S

# Let's test the implementation. Conveniently, PICOS uses swap operators
# internally for computing the transpose of affine expressions.
d = 8
S_ref_sp = pc.expressions.data.cvxopt_K(d, d)    # as a CVXOPT sparse matrix
S_ref_np = pc.expressions.data.cvx2np(S_ref_sp)  # as a NumPy dense array
S_sol_np = np.array(swap(d))

if np.allclose(S_sol_np, S_ref_np):
    print("Good job!")
else:
    print("S(d) is not the swap operator :-(")


---

Recall that we want to compute ${\lVert \mathcal{D} \rVert}_\diamond$ where
$\mathcal{D} = \frac{1}{2} \left( \mathcal{B}_1 + \mathcal{B}_2 \right)$. A
pair of primal and dual SDP formulations for ${\lVert \mathcal{D}
\rVert}_\diamond$ is the following ([Watrous, 2009](#references)):

$$
\begin{align*}
    \text{maximize} ~&~ \Re(\operatorname{Tr}(\mathcal{J}_{BA}(\mathcal{D}) X))
        \tag{primal} \\
    \text{subject to}
    ~&~ \begin{bmatrix}
        I_B \otimes \varrho_A & X \\
        X^\dagger & I_B \otimes \sigma_A
    \end{bmatrix} \succeq 0, \\
    ~&~ \operatorname{Tr}(\varrho_A) = 1,\\
    ~&~ \operatorname{Tr}(\sigma_A) = 1,\\
    \text{where}
    ~&~ \varrho_A, \sigma_A \in \mathbb{C}^{d \times d}~\text{Hermitian}, \\
    ~&~ X \in \mathbb{C}^{d^2 \times d^2},\\
\end{align*}
$$

and

$$
\begin{align*}
    \text{minimize} ~&~ \frac{1}{2} \left(
        {\lVert \operatorname{Tr}_B(M_{AB}) \rVert}_\infty +
        {\lVert \operatorname{Tr}_B(N_{AB}) \rVert}_\infty
    \right) \tag{dual1} \\
    \text{subject to}
    ~&~ \begin{bmatrix}
        M_{AB} & -\mathcal{J}_{BA}(\mathcal{D}) \\
        -{\mathcal{J}_{BA}(\mathcal{D})}^\dagger & N_{BA}
    \end{bmatrix} \succeq 0, \\
    \text{where}
    ~&~ M_{AB}, N_{AB} \in \mathbb{C}^{d^2 \times d^2}~\text{Hermitian}, \\
\end{align*}
$$

where ${\lVert \cdot \rVert}_\infty$ denotes the spectral norm, which for HPSD
operands equals the largest eigenvalue. For this reason, the dual can be
simplified to

$$
\begin{align*}
    \text{minimize} ~&~ \frac{1}{2} (\mu + \nu) \tag{dual2} \\
    \text{subject to}
    ~&~ \begin{bmatrix}
        M_{AB} & -\mathcal{J}_{BA}(\mathcal{D}) \\
        -{\mathcal{J}_{BA}(\mathcal{D})}^\dagger & N_{BA}
    \end{bmatrix} \succeq 0, \\
    ~&~ \operatorname{Tr}_B(M_{BA}) \preceq \mu I_A,\\
    ~&~ \operatorname{Tr}_B(N_{BA}) \preceq \nu I_A,\\
    \text{where}
    ~&~ M_{AB}, N_{AB} \in \mathbb{C}^{d^2 \times d^2}~\text{Hermitian}, \\
    ~&~ \mu, \nu \in \mathbb{R}.\\
\end{align*}
$$

Watrous ([2009](#references)) could show that both conic programs are
essentially strictly feasible (i.e., apart from linear equalities, the feasible
regions of both problems have nonempty interior). As a result, strong duality
holds: the optimal values of the primal and dual program coincide. We will use
this to verify correctness of our implementation.


---

<div class="alert alert-info">

   **Task 4.4 (grand finale):** Implement (primal) and either (dual1) or
   (dual2) for the given Werner-Holevo channels, $\mathcal{B}_1$ and
   $\mathcal{B}_2$, and for $t = \frac{1}{2}.$

   <details>
   <summary style='display: list-item'>Hint</summary>

   - By linearity, it is

     $$
     \begin{align*}
         \mathcal{J}_{BA}(\mathcal{D})
         &= \mathcal{J}_{BA}\left(
             \frac{1}{2}\left( \mathcal{B}_1 + \mathcal{B}_2 \right) \right)
         = \frac{1}{2}\left(
             \mathcal{J}_{BA}(\mathcal{B}_1) + \mathcal{J}_{BA}(\mathcal{B}_2)
             \right).
     \end{align*}
     $$

   </details>

   <details>
   <summary style='display: list-item'>PICOS functions</summary>

   | on paper | in picos |
   | --- | --- |
   | $$X \in \mathbb{C}^{a \times b}$$ | `X = pc.ComplexVariable("X", (a, b))` |
   | $$X = X^\dagger \in \mathbb{C}^{d \times d}$$ | `X = pc.HermitianVariable("X", d)` |
   | $$I_d$$ | `pc.I(d)` |
   | $$A \otimes B$$ | `A @ B` |
   | $$A / \alpha$$ | `A/alpha` |
   | $$A^\dagger$$ | `A.H` |
   | $$\Re(A),~\Im(A)$$ | `A.real` / `A.imag` |
   | $$\operatorname{Tr}(A)$$ | `A.tr` |
   | $$\operatorname{Tr}_A(\varrho_{AB}),~\operatorname{Tr}_B(\varrho_{AB})$$ (for $d_A = d_B = 2$) | `rho.tr0` / `rho.tr1` |
   | $$\operatorname{Tr}_\cdots(\varrho_{\cdots})$$ (for any subsystems) | `rho.partial_trace(...)` [[docs](https://picos-api.gitlab.io/picos/api/picos.expressions.exp_biaffine.html#picos.expressions.exp_biaffine.BiaffineExpression.partial_trace)] |
   | $${\lVert A \rVert}_\infty$$ | `pc.SpectralNorm(A)` |
   | $$\text{Let}~A = \ldots$$ | `A = (...).rename("A")` | 

   </details>
</div>

In [ ]:
d = 2

# TODO: Define constants used in both problems: S, I, J(B_1), J(D), ...

# TODO: Define and solve the primal problem P. (Unicode for var names: ρ σ)
P = pc.Problem("Primal")

print(f"Primal optimum:      {P.value}")

# TODO: Define and solve a dual problem D. (Unicode for var names: μ 𝜈)
D = pc.Problem("Dual")

print(f"Dual optimum:        {D.value}")

# Check if both solutions are numerically equal.
print(f"Primal-dual gap:     {abs(P.value - D.value)}")
assert np.allclose(P.value, D.value), "Primal and dual solution are not close."

# Compute the success probability s.
diag_norm_D = (P.value + D.value) / 2
s = (1 + diag_norm_D) / 2

print(f"Success probability: {s}")

If everything worked out, then the primal-dual gap should be close to zero and
Alice's success probability $s^*$ should be a value between $\frac{1}{2}$ and
$1$.

In the bonus task, you can contrast this with an exact solution for the
probability $q^*$ of guessing Bob's chosen channel without an $R$-subsystem. If
this probability is significantly smaller than $s^*$, then this proves that an
auxiliary system—and by extension entanglement—are useful tools for channel
discrimination.

---

<div class="alert alert-info">

   **Bonus task 4.5:** Compute

   $$
      {\lVert \mathcal{D} \rVert}_1
      = \sup_{X \colon {\lVert X \rVert}_1 \leq 1}
         {\lVert \mathcal{D}(X) \rVert}_1
      \qquad
      \text{and}
      \qquad
      q^*
   $$

   for $d = 2$ by following the argumentation below.

</div>

First, we use without proof (see Watrous ([2018](#references))) the observation
that the supremum is attained by the density operator of a pure state, formally

$$
   \sup_{X \colon {\lVert X \rVert}_1 \leq 1}
      {\lVert \mathcal{D}(X) \rVert}_1
   = \sup_{x \colon {\lVert x \rVert}_2 = 1}
      {\left\lVert \mathcal{D}(xx^\dagger) \right\rVert}_1.
$$

Further, since ${\lVert x \rVert}_2 = {\lVert \overline{x} \rVert}_2$,

$$
   {\lVert \mathcal{D} \rVert}_1
   = \sup_{x \colon {\lVert x \rVert}_2 = 1}
      {\left\lVert \mathcal{D}{(xx^\dagger)}^T \right\rVert}_1.
$$

Next, we compute $\mathcal{D}(xx^\dagger)^T$ for $\lVert x \rVert_2 = 1$ and $d
 = 2$:

<div hidden>

$$
\providecommand{\TODO}{}\renewcommand{\TODO}{{\color{red}{[\ldots]}}}
$$

</div>

$$
\begin{align*}
    {\mathcal{D}(xx^\dagger)}^T
    &= \frac{1}{2} {\left(
        \mathcal{B}_1(xx^\dagger) - \mathcal{B}_2(xx^\dagger) \right)}^T \\
    &= \frac{1}{2} \left( \frac{1}{d + 1} \left(
        \operatorname{Tr}(xx^\dagger) I + xx^\dagger \right) 
        - \frac{1}{d - 1} \left(
        \operatorname{Tr}(xx^\dagger) I - xx^\dagger \right)
        \right) \\
    &= \underbrace{\frac{2}{3} \left(
            \TODO
        \right)}_{Z}.
\end{align*}
$$

After simplifying, within the parenthesis there should remain a sum of two
rather simple products.

Next, we use the definition of the trace norm and the fact that $\sqrt{I} = I$:

<div hidden>

$$
\providecommand{\TODO}{}\renewcommand{\TODO}{{\color{red}{[\ldots]}}}
$$

</div>

$$
\begin{align*}
    {\left\lVert {\mathcal{D}(xx^\dagger)}^T \right\rVert}_1
    &= \operatorname{Tr}\left(\sqrt{ Z^\dagger Z }\right) \\
    &= \TODO
\end{align*}
$$

The result should be a constant independent of $x$. This immediately yields

<div hidden>

$$
\providecommand{\TODO}{}\renewcommand{\TODO}{{\color{red}{[\ldots]}}}
$$

</div>

$$
\begin{align*}
    {\lVert \mathcal{D} \rVert}_1 =
        \TODO
    \quad\text{and}\quad
    q^* =
        \TODO.
\end{align*}
$$

---

### Additional resources

- The problem above is one of the five "starter pieces" presented by Siddhu and
  Tayur ([2022](#references)) as an introduction to semidefinite programming for
  quantum information, who discuss it in greater detail.

### Acknowledgements

This exercise is based on Problem 3 of Siddhu and Tayur ([2022](#references))
and the accompanying Python/PICOS notebook, available at
https://github.com/vsiddhu/SDP-Quantum-OR.

## References

- Michel X. Goemans and David P. Williamson, 2001. Approximation algorithms for
  MAX-3-CUT and other problems via complex semidefinite programming. In:
  *Proceedings of the thirty-third annual ACM symposium on Theory of computing*,
  pp. 443-452. ACM. https://doi.org/10.1145/380752.380838
- Vikesh Siddhu and Sridhar Tayur, 2022. Five starter pieces: Quantum
  information science via semi-definite programs. *INFORMS TutORials in
  Operations Research.* https://doi.org/10.1287/educ.2022.0243 (preprint:
  https://arxiv.org/abs/2112.08276)
- John Watrous, 2009. Semidefinite programs for completely bounded norms.
  *Theory of Computing*, 5, 11, pp. 217-238.
  https://doi.org/10.4086/toc.2009.v005a011 (preprint:
  https://arxiv.org/abs/0901.4709)
- John Watrous, 2018. *The Theory of Quantum Information.* Cambridge University
  Press. https://doi.org/10.1017/9781316848142 (preprint:
  https://cs.uwaterloo.ca/~watrous/TQI/)